Installing the Julep Client

In [1]:
!pip install --upgrade julep --quiet

In [2]:
import uuid

# NOTE: these UUIDs are used in order not to use the `create_or_update` methods instead of
# the `create` methods for the sake of not creating new resources every time a cell is run.
AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

## creating julep client with the api key


In [3]:
from julep import Client
import os

JULEP_API_KEY = os.environ.get("JULEP_API_KEY")

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

### Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the Agent section in [Julep Concepts](https://docs.julep.ai/docs/concepts/agents).

In [4]:
# Create agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Spiderman",
    about="AI that can crawl the web and extract data",
    model="gpt-4o",
)

### Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/tasks).

In [218]:
import yaml

cloudinary_api_key = os.environ["CLOUDINARY_API_KEY"]
cloudinary_api_secret = os.environ["CLOUDINARY_API_SECRET"]
cloudinary_cloud_name = os.environ["CLOUDINARY_CLOUD_NAME"]

# Define the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/src/schemas/create_task_request.json
name: Julep Video Processing With Natural Language
description: Process a video using natural language instructions

########################################################
####################### INPUT SCHEMA ###################
########################################################
input_schema:
  type: object
  properties:
    video_url:
      type: string
      description: The url of the file to upload
    public_id:
      type: string
      description: The public id of the file to upload
    transformation_prompt:
      type: string
      description: The prompt for the transformations to apply to the file

########################################################
####################### TOOLS ##########################
########################################################
                          
# Define the tools that the task will use in this workflow
tools:
- name: cloudinary_upload
  type: integration
  integration:
    provider: cloudinary
    method: media_upload
    setup:
      cloudinary_api_key: "{cloudinary_api_key}"
      cloudinary_api_secret: "{cloudinary_api_secret}"
      cloudinary_cloud_name: "{cloudinary_cloud_name}"

########################################################
####################### MAIN WORKFLOW ##################
########################################################

main:
# Step 0: Upload the video to cloudinary
- tool: cloudinary_upload
  arguments:
    file: $ steps[0].input.video_url
    public_id: $ steps[0].input.public_id
    upload_params:
      resource_type: video

# Step 1: Generate the transformation json
- prompt:
  - role: user
    content:

      - type: text
        text: |-
          You are a Cloudinary expert. You are given a medial url. it might be an image or a video.
          You need to come up with a json of transformations to apply to the given media.
          Overall the json could have multiple transformation json objects.
          Each transformation json object can have the multiple key value pairs.
          Each key value pair should have the key as the transformation name like "aspect_ratio", "crop", "width" etc and the value as the transformation parameter value.
          Note: Provide the image url as it is when overlaying images.
          Note: When you use overlay with an image url, you need to add a nested json containing url: <image_url>
          Given below is an example of some key value pairs that you might need to use when constructing the json.
            "aspect_ratio": "1.0",
              "width": "250",
              "fetch_format": "auto"
              "overlay":
                "url": "<image_url>"
              "flags": "layer_apply"
            
         
      - type: image_url
        image_url:
          url: $ _.url

      - type: text
        text: |-
          $ f'''Hey, check the video above, I need to apply the following transformations using cloudinary.
          {{steps[0].input.transformation_prompt}}'''

  unwrap: true
  settings:
    model: gemini/gemini-1.5-pro

# Step 2: Extract the json from the model's response
- evaluate:
    model_transformation: >-
      $ extract_json(_)

# Step 3: Upload the video to cloudinary
- tool: cloudinary_upload
  arguments:
    file: $ steps[0].input.video_url
    public_id: $ steps[0].input.public_id
    upload_params:
      transformation: $ _.model_transformation
      resource_type: video

# Step 4: Return the transformed video url
- evaluate:
    transformed_video_url: $ _.url
""")

In [219]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

To learn more about executions, please refer to the `Executions` section in [Julep Concepts](https://docs.julep.ai/docs/concepts/execution).

In [220]:
# creating an execution object
transformation_prompt = """
1- I want to add an overlay an the following image to the video, and apply a layer apply flag also. Here's the image url:
https://avatars.githubusercontent.com/u/112750682?s=200&v=4

2- I also want you to to blur the video, and add a fade in and fade out effect to the video with a duration of 3 seconds each.
"""

input_video_url = "http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerMeltdowns.mp4"

execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "video_url":  input_video_url,
        "public_id": "video_test2",
        "transformation_prompt": transformation_prompt,
    }
)
execution.id

'067ffb5c-1240-7967-8000-a40019c3084d'

## Checking execution details and output

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:olive;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [221]:
# Get execution details
execution = client.executions.get(execution.id)
# Print the output
print(execution.output)

{'url': 'https://res.cloudinary.com/dpnjjk8mb/video/upload/v1738934100/video_test2.mp4', 'base64': None, 'meta_data': {'url': 'http://res.cloudinary.com/dpnjjk8mb/video/upload/v1738934100/video_test2.mp4', 'tags': [], 'type': 'upload', 'audio': {'codec': 'aac', 'bit_rate': '191999', 'channels': 2, 'frequency': 44100, 'channel_layout': 'stereo'}, 'bytes': 2252313, 'video': {'codec': 'h264', 'level': 31, 'profile': 'High', 'bit_rate': '1002377', 'time_base': '1/48', 'pix_format': 'yuv420p'}, 'width': 1280, 'format': 'mp4', 'height': 720, 'api_key': '518455844981529', 'version': 1738934100, 'asset_id': '069d70c84e88ee9293a1d753b3ca3898', 'bit_rate': 1197518, 'duration': 15.046531, 'existing': True, 'is_audio': False, 'rotation': 0, 'nb_frames': 361, 'signature': '836eddb515d796dc8484d7dc108cb67fbe0ba1d5', 'created_at': '2024-11-20T08:55:44Z', 'frame_rate': 24.0, 'version_id': '85846ee1c860645e1149e8029051c07d', 'placeholder': False, 'asset_folder': '', 'display_name': 'video_test2', 'play

In [242]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

if transitions[0].type == "finish":
    transformed_video_url = transitions[0].output['transformed_video_url']

Transition type:  init
Transition output:  {'public_id': 'video_test2', 'video_url': 'http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/ForBiggerMeltdowns.mp4', 'transformation_prompt': "\n1- I want to add an overlay an the following image to the video, and apply a layer apply flag also. Here's the image url:\nhttps://avatars.githubusercontent.com/u/112750682?s=200&v=4\n\n2- I also want you to to blur the video, and add a fade in and fade out effect to the video with a duration of 3 seconds each.\n"}
--------------------------------------------------
Transition type:  step
Transition output:  {'url': 'https://res.cloudinary.com/dpnjjk8mb/video/upload/v1738934100/video_test2.mp4', 'base64': None, 'meta_data': {'url': 'http://res.cloudinary.com/dpnjjk8mb/video/upload/v1738934100/video_test2.mp4', 'tags': [], 'type': 'upload', 'audio': {'codec': 'aac', 'bit_rate': '191999', 'channels': 2, 'frequency': 44100, 'channel_layout': 'stereo'}, 'bytes': 2252313, 'video': {'codec': 

## Related Concepts

- [Agents](https://docs.julep.ai/concepts/agents)
- [Tasks](https://docs.julep.ai/concepts/tasks)
- [Tasks](https://docs.julep.ai/concepts/tasks)lep.ai/concepts/tasks)